In [0]:
%sh
vmstat 3

In [0]:
orders_sdf = spark.read.csv('/FileStore/tables/orders.csv', header=True, inferSchema=True)
trains_sdf = spark.read.csv('/FileStore/tables/order_products_train.csv', header=True, inferSchema=True)
products_sdf = spark.read.csv('/FileStore/tables/products.csv', header=True, inferSchema=True)
aisles_sdf = spark.read.csv('/FileStore/tables/aisles.csv', header=True, inferSchema=True)
depts_sdf = spark.read.csv('/FileStore/tables/departments.csv', header=True, inferSchema=True)

300MB 이상의 파일을 Databricks cloud storage에 upload 되지 않으므로 order_products_prior.zip 파일을 압축된 채 그대로 로드하고, 이를 pandas의 read_csv()의 option인 compressed를 이용하여 pandas DataFrame으로 로드한 뒤 다시 spark DataFrame으로 변환

In [0]:
%fs 
cp /FileStore/tables/order_products_prior.zip file:/home/order_products_prior.zip 

In [0]:
%sh
ls -lia /home

In [0]:
import pandas as pd

priors_pdf = pd.read_csv('/home/order_products_prior.zip', compression='zip', header=0, sep=',', quotechar='"')
priors_sdf = spark.createDataFrame(priors_pdf)

In [0]:
del priors_pdf # 메모리 절약을 위해 pandas dataframe삭제 
#gc.collect() 는 수행하지 말것. 수행시간이 오래 걸림.

In [0]:
sdf_dict = {"orders":orders_sdf, "order_product_train":trains_sdf, "order_product_prior":priors_sdf, 
            "product":products_sdf, "aisles":aisles_sdf, "departments":depts_sdf
           }
for sdf_name, sdf in sdf_dict.items():
    print("########## ", sdf_name, "schema ###########")
    print(sdf.printSchema())

In [0]:
# 로드한 DataFrame을 임시 table/view로 생성. 
orders_sdf.createOrReplaceTempView("orders")
priors_sdf.createOrReplaceTempView("priors")
trains_sdf.createOrReplaceTempView("trains")
products_sdf.createOrReplaceTempView("products")
aisles_sdf.createOrReplaceTempView("aisles")
depts_sdf.createOrReplaceTempView("depts")

In [0]:
# temporary table/view로 등록되어 있는지 확인. 
spark.catalog.listTables()

#### orders 테이블 
개별 user_id로 prior는 여러개의 order_id를 가지나 train과 test는  단 1개의 order_id를 가짐. 
즉 eval_set가 train인 경우는 user_id와 order_id가 1:1의 관계임
* order_id: 주문번호
* user_id: 고객번호
* eval_set : order 데이터의 prior/train/test 구분
* order_number: 개별 사용자별 주문 순번
* order_dow: 주문 요일을 숫자로 나타냄(토~금)
* order_hour_of_day: 주문 시간을 숫자로 나타냄(0~23)
* days_since_prior_order: 이전 주문 이후 현 주문까지 걸린 일자. 첫 주문은 Null

#### order_product_prior
개별 주문의 상품 정보. 과거 주문의 상품 정보 가짐. 
* order_id: 주문번호
* product_id: 상품번호
* add_to_cart_order: 개별 주문에서 상품을 cart에 담은 순서
* reordered: 개별 사용자 별 해당 상품에 대한 재 주문 여부. 1은 재 주문, 0은 첫 주문. 학습 모델의 target 속성 임. 

#### order_product_train
개별 주문의 상품 정보. 개별 사용자당 한개의 주문 정보. 
* order_id: 주문번호
* product_id: 상품번호
* add_to_cart_order: 개별 주문에서 상품을 cart에 담은 순서
* reordered: 개별 사용자 별 해당 상품에 대한 재 주문 여부. 1은 재 주문, 0은 첫 주문. 학습 모델의 target 속성 임.

In [0]:
%sql
select * from orders limit 10
/*
select * from orders limit 10

-- 개별 user가 주문한 여러건의 주문 데이터를 가지고 있음. 첫 주문의 경우 days_since_prior_order가 null임. 
-- eval_set은 해당 주문의 주문 상품 데이터가 priors에 있는지, trains/test에 있는지를 나타냄
-- 개별 user의 여러건의 주문 데이터 중, 가장 최근 1건의 order만 trains/test에 속함. 나머지 건은 모두 priors에 속함.  
select * from orders where user_id = 1 

-- 개별 주문의 여러건의 과거 상품 주문 데이터를 가지고 있음. 개별 user가 주문한 여러건의 주문 정보를 가짐. reordered가 해당 상품에 대한 재주문 여부이며 1이 재주문, 0이 첫주문 임. 
select * from priors limit 10

-- user_id 1의 order_id 2398795의 과거 상품 주문 정보 확인. order_id 2539329는 첫주문이므로 priors의 reordered는 모두 0
select * from priors where order_id = 2398795 

-- 개별 주문의 여러건의 상품 주문 데이터를 가지고 있음. 개별 user가 주문한 1건의 주문정보를 가짐. 
select * from trains limit 10

-- order_id 1은 user_id 112108 을 가지고 있으며 user_id 112108은 여러건의 주문을 가짐. 
select * from orders where user_id = (select user_id from orders where order_id=1) 
select * from trains where order_id = 1

*/

In [0]:
%sql
-- 상품에 대한 정보를 가짐. aisle_id는 상품 중분류, department_id는 상품 대분류. 마트의 복도 선반별(aisles), 특정 층또는 구역(departments)
select a.*, b.aisle, c.department 
from products a, aisles b, depts c
where a.aisle_id = b.aisle_id and a.department_id = c.department_id
/* 
select * from products 

select * from aisles

select * from depts

select a.*, b.aisle, c.department 
from products a, aisles b, depts c
where a.aisle_id = b.aisle_id and a.department_id = c.department_id
*/

In [0]:
%sql -- format sql을 이용하여 formatting 할 수 있음.
select
  a.*,
  b.aisle,
  c.department
from
  products a,
  aisles b,
  depts c
where
  a.aisle_id = b.aisle_id
  and a.department_id = c.department_id

In [0]:
%sql
-- 주요 테이블의 건수 확인
SELECT 'priors count' as table_name_count, COUNT(*) cnt FROM priors
union all
select 'trains count', count(*) cnt from trains
union all
SELECT 'orders count', COUNT(*) cnt FROM orders

In [0]:
%sql
-- eval_set별 주문 건수 확인
select eval_set, count(*) from orders group by eval_set


In [0]:
%sql
-- train은 1 user에 1 order이므로 distinct id count가 동일. 
-- prior는 train과 test에 해당하는 user를 가지고 있음. 
-- test도 1 user에 1 order임. 
select eval_set, count(distinct user_id) from orders group by eval_set

In [0]:
%sql

select user_id, count(*) cnt from orders group by user_id

In [0]:
%sql
-- orders의 user_id 별로 대략적인 분포도를 확인
select max(cnt), min(cnt), avg(cnt), count(*) 
from (
  select user_id, count(*) as cnt from orders group by user_id
)

In [0]:
%sql
-- user_id 건수가 많은 순으로 1000개만 출력
select user_id, count(*) cnt from orders group by user_id order by 2 desc limit 1000

In [0]:
%sql
-- 이전 SQL은 너무 많은 데이터를 클라이언트로 가져 오므로 Resource 소모가 심함. 
-- 아래와 같이 histogram 구하는 SQL로 변경. 
-- With as 절을 사용. 
with 
user_cnt_group as
(
select user_id, count(*) user_cnt from orders group by user_id
)
-- end of with
select user_cnt_bin||'-'||(user_cnt_bin+10) as user_cnt_bin_range, bin_count
from (
  select floor(user_cnt/10.0)*10 as user_cnt_bin, count(*) bin_count 
  from user_cnt_group
  group by floor(user_cnt/10.0)*10
) order by bin_count desc

/*
select user_cnt_bin||'-'||(user_cnt_bin+10) as user_cnt_bin_range, bin_count
from (
  select floor(user_cnt/10.0)*10 as user_cnt_bin, count(*) bin_count 
  from (
    select user_id, count(*) user_cnt from orders group by user_id
  )
  group by floor(user_cnt/10.0)*10
) order by bin_count desc
*/

In [0]:
%sql
-- 상품별 건수의 대략적인 분포도를 확인. 
select gubun, max(cnt), min(cnt), avg(cnt)
from (
  select 'prior' as gubun, product_id, count(*) as cnt from priors group by  product_id
  union all
  select 'train' as gubun, product_id, count(*) from trains group by product_id
) group by gubun

In [0]:
%sql
-- priors 테이블과 trains 테이블을 함께 조사하기 위해 두 테이블을 합침. 
drop table if exists priors_trains;



In [0]:
%fs
rm -r dbfs:/user/hive/warehouse/priors_trains

In [0]:
%sql

create table priors_trains
as
select 'train' as gubun, a.* from trains a
union all
select 'prior' as gubun, a.* from priors a

In [0]:
spark.catalog.listTables()

In [0]:
%sh
vmstat 3

In [0]:
%sql
-- 메모리에 여유가 있으면 수행 속도 향상을 위해 cache로 로드 
cache table priors_trains
-- uncache table order_prods

### 상품별 전체 주문 건수, 재주문 건수, 재주문 비율 조사
* 상품별 전체 주문 건수와 재주문 건수를 구하여 재 주문 비율이 어느정도 되는지 확인. 
* 일부 상품의 경우 주문 건수가 너무 작아서 재주문 또는 첫주문 비율이 너무 높게 나옴. 
* 상품별 재주문 비율과 상품 중분류/대분류 레벨의 재주문 비율 비교도 조사.

In [0]:
%sql
-- 주문 상품 건수가 많은 상위 10건의 product 추출. 
select product_id, count(*) cnt from priors_trains group by product_id order by 2 desc limit 10

In [0]:
%sql
select * from priors_trains;

In [0]:
%sql
-- product_id로는 어떤 상품이 주문이 많이 되었는지 알수 없으므로 product_name을 얻기 위해 product table과 조인. 
with 
order_prods_grp as
(
select gubun, product_id, count(*) cnt 
from priors_trains -- trains, priors 
group by gubun, product_id
)
-- end of with 절
select a.*, b.product_name 
from order_prods_grp a, products b 
where a.product_id = b.product_id
order by a.cnt desc limit 1000

In [0]:
%sql
-- 상품 주문 건수를 상품 중분류, 상품 대분류 별로도 알아 보기 위해 product_id로 group by 한 결과에 aisles 테이블과 depts 테이블을 조인. 
with order_prods_grp as
(
select gubun, product_id, count(*) cnt 
from priors_trains -- trains, priors 
group by gubun, product_id
)
-- end of with 절
select a.*, b.product_name, c.aisle, d.department
from order_prods_grp a, products b, aisles c, depts d 
where a.product_id = b.product_id 
and b.aisle_id = c.aisle_id
and b.department_id = d.department_id
order by a.cnt desc limit 1000
/*
상품 대분류 레벨로 상품 주문건수를 알아보려면 아래와 같이 department_id(또는 중분류는 aisle_id)로 group by 를 수행하여야 하나, 시각화를 databricks 노트북을 이용하여 할 경우 product_id 레벨로 주문 건수를 대분류/중분류 레벨로 변경하면 자동으로 Graph에서 대분류/중분류 레벨로 합산하여 보여주게 되므로 group by를 product_id 레벨로 해도 대분류/중분류 레벨의 건수를 알 수 있음. 

with order_depts_grp as
(
select b.department_id, count(*) as cnt
from priors_trains a, products b 
where a.product_id = b.product_id 
group by gubun, b.department_id
)
select a.*, b.department
from order_depts_grp a, depts b
where a.department_id = b.department_id
order by cnt desc limit 10
*/


In [0]:
# 보다 상세한 시각화를 위해서 Spark DataFrame에서 Pandas DataFrame으로 변환. 
# sql 결과를 pandas dataframe으로 변환. 
# spark.sql(query)로 query문장을 수행하여 결과를 Spark DataFrame으로 반환
# 이때 query는 """ 로 감싸져야 한다. 
query = """
with order_prods_grp as
(
select gubun, product_id, count(*) cnt 
from priors_trains -- trains, priors 
group by gubun, product_id
)
select a.*, b.product_name, c.aisle, d.department
from order_prods_grp a, products b, aisles c, depts d 
where a.product_id = b.product_id 
and b.aisle_id = c.aisle_id
and b.department_id = d.department_id
order by a.cnt desc limit 1000
"""
#query를 수행하여 Spark DataFrame으로 반환. 
grp_result_sdf = spark.sql(query)

# Spark DataFrame을 pandas DataFrame으로 변환. 
grp_result_pdf = grp_result_sdf.select('*').toPandas()

display(grp_result_sdf)

In [0]:
# pandas dataframe을 databricks의 내장 graph chart 기능으로 시각화
display(grp_result_pdf)

In [0]:
# Pandas DataFrame의 plot.bar 기능을 이용하여 bar chart 시각화
import pandas as pd
import matplotlib.pyplot as plt

# 상위 40개만 bar chart로 표현. 
plt.figure(figsize=(10, 8))
grp_result_pdf.iloc[:40, :].plot.bar(x='aisle', y='cnt', rot=90)
plt.show()

In [0]:
import seaborn as sns

#seaborn을 사용하여 bar chart 시각화
sns.barplot(x="aisle", y="cnt", data=grp_result_pdf.iloc[:40,]) #estimator=sum
plt.xticks(rotation=90)
plt.show()

In [0]:
%sql
select gubun, count(*) from priors_trains group by gubun

#### 상품별 reordered 여부에 따른 건수, 비율 분석

In [0]:
%sql
-- group by를 product_id, reordered 레벨로 수행하여 개별 상품별로 reordered여부에 따른 건수 확인.  
-- case when then else end 문을 group by의 aggregation 함수에 적용
with 
order_prods_grp as
(
select gubun, product_id, reordered, count(*) cnt
from priors_trains -- trains, priors 
group by gubun, product_id, reordered -- group by를 gubun, product_id, reordered 로 적용
)
-- end of with 절
select a.*, b.product_name, c.aisle, d.department
from order_prods_grp a, products b, aisles c, depts d 
where a.product_id = b.product_id 
and b.aisle_id = c.aisle_id
and b.department_id = d.department_id
order by a.cnt desc limit 100

In [0]:
%sql
-- 상품별 주문 건수 대비 reorder/no reorder 건수의 비율 구하기.
-- 특정 상품들은 주문 건수가 작아서 reorder/no reorder 비율이 아주 작거나 아주 큼 

with 
-- 상품별 reorder, no reorder및 전체 건수 구하기
order_prods_grp as
(
select product_id 
  , sum(case when reordered=1 then 1 else 0 end) as reordered_cnt -- 상품별 재주문 건수
  , sum(case when reordered=0 then 1 else 0 end) as no_reordered_cnt -- 상품별 재 주문 하지 않은 건수 
  , count(*) prd_total_cnt -- 상품별 건수 
from priors_trains -- trains, priors 
group by product_id -- group by product_id having total_cnt > 100
)
-- end of with 절.
-- 상품별 전체 주문 건수 대비 reorder 및 no reorder 건수 비율 구하기
select a.*
  , reordered_cnt/prd_total_cnt as reordered_pct -- 상품별 재 주문 비율
  , no_reordered_cnt/prd_total_cnt no_reordered_pct -- 상품별 재주문 하지 않은 비율
  , b.product_name, c.aisle_id, c.aisle, d.department_id, d.department
from order_prods_grp a, products b, aisles c, depts d 
where a.product_id = b.product_id 
and b.aisle_id = c.aisle_id
and b.department_id = d.department_id
order by reordered_pct desc limit 1000

In [0]:
query = """
with 
-- 상품별 reorder, no reorder및 전체 건수 구하기
order_prods_grp as
(
select product_id 
  , sum(case when reordered=1 then 1 else 0 end) as reordered_cnt -- 상품별 재주문 건수
  , sum(case when reordered=0 then 1 else 0 end) as no_reordered_cnt -- 상품별 재 주문 하지 않은 건수 
  , count(*) prd_total_cnt -- 상품별 건수 
from priors_trains -- trains, priors 
group by product_id -- group by product_id having total_cnt > 100
)
-- end of with 절.
-- 상품별 전체 주문 건수 대비 reorder 및 no reorder 건수 비율 구하기
select a.*
  , reordered_cnt/prd_total_cnt as reordered_pct -- 상품별 재 주문 비율
  , no_reordered_cnt/prd_total_cnt no_reordered_pct -- 상품별 재주문 하지 않은 비율
  , b.product_name, c.aisle_id, c.aisle, d.department_id, d.department
from order_prods_grp a, products b, aisles c, depts d 
where a.product_id = b.product_id 
and b.aisle_id = c.aisle_id
and b.department_id = d.department_id
"""
grp_result_sdf = spark.sql(query)
grp_result_pdf = grp_result_sdf.select('*').toPandas()

In [0]:
sns.scatterplot(data=grp_result_pdf, x="reordered_pct", y="reordered_cnt")

In [0]:
%sql
-- 상품별 주문 건수 대비 reorder/no reorder 건수의 비율 구하기.
-- 특정 상품들은 주문 건수가 작아서 reorder/no reorder 비율이 아주 작거나 아주 큼 
with 
-- 상품별 reorder, no reorder및 전체 건수 구하기
order_prods_grp as
(
select product_id 
  , sum(case when reordered=1 then 1 else 0 end) as reordered_cnt -- 상품별 재주문 건수
  , sum(case when reordered=0 then 1 else 0 end) as no_reordered_cnt -- 상품별 재 주문 하지 않은 건수 
  , count(*) prd_total_cnt -- 상품별 건수 
from priors_trains -- trains, priors 
group by product_id -- group by product_id having total_cnt > 100
)
-- end of with 절.
-- 상품별 전체 주문 건수 대비 reorder 및 no reorder 건수 비율 구하기
select a.*
  , reordered_cnt/prd_total_cnt as reordered_pct -- 상품별 재 주문 비율
  , no_reordered_cnt/prd_total_cnt no_reordered_pct -- 상품별 재주문 하지 않은 비율
  , b.product_name, c.aisle_id, c.aisle, d.department_id, d.department
from order_prods_grp a, products b, aisles c, depts d 
where a.product_id = b.product_id 
and b.aisle_id = c.aisle_id
and b.department_id = d.department_id
-- 재 주문 건수가 100000건 이상이면서 재주문 비율이 0.7 이상인 상품 추출. 
and reordered_cnt > 100000 and reordered_cnt/prd_total_cnt > 0.7

In [0]:
%sql
-- 상품별 주문 건수 대비 reorder/no reorder 건수의 비율 구하기. 
-- 상품별 reorder, no reorder및 전체 건수 구하기
with 
order_prods_grp as
(
select product_id 
  , sum(case when reordered=1 then 1 else 0 end) as reordered_cnt -- 상품별 재주문 건수
  , sum(case when reordered=0 then 1 else 0 end) as no_reordered_cnt -- 상품별 재주문 하지 않은 건수
  , sum(reordered) as reordered_cnt_01 -- reordered는 0과 1이므로 상품별 재 주문 건수는 sum(reordered)로도  가능.
  , avg(reordered) as avg_reordered -- reordered는 0과 1이므로 상품별 재 주문 비율은 avg(reordered)로도 가능.
  , count(*) prd_total_cnt -- 상품별 건수 
from priors_trains -- trains, priors 
group by product_id -- group by product_id having total_cnt > 100
)
-- end of with 절.
-- 상품별 전체 주문 건수 대비 reorder 및 no reorder 건수 비율 구하기
select a.*
  , reordered_cnt/prd_total_cnt as reordered_pct -- 상품별 재 주문 비율
  , no_reordered_cnt/prd_total_cnt no_reordered_pct -- 상품별 재주문 하지 않은 비율
  , b.product_name, c.aisle_id, c.aisle, d.department_id, d.department
from order_prods_grp a, products b, aisles c, depts d 
where a.product_id = b.product_id 
and b.aisle_id = c.aisle_id
and b.department_id = d.department_id

In [0]:
%sql
-- aisle별 주문 reorder의 건수와 비율을 구함. 
-- alias로 바로 group by 할 수 있으나 이후 join등을 위해 product_id로 먼저 group by 한 결과를 다시 alias 로 group by sum()하여 구함. 
with 
-- 상품별 reorder, no reorder 건수 구하기
order_prods_grp as
(
select product_id 
  , sum(case when reordered=1 then 1 else 0 end) as reordered_cnt -- 상품별 재주문 건수
  , sum(case when reordered=0 then 1 else 0 end) as no_reordered_cnt -- 상품별 재주문 하지 않은 건수 
  , count(*) prd_total_cnt -- 상품별 건수 
from priors_trains -- trains, priors 
group by product_id -- group by product_id having total_cnt > 100
), -- end of order_prods_grp
-- 두번째 집합 order_prods_grp_detail
order_prods_grp_detail as
(
select a.* 
  , reordered_cnt/prd_total_cnt as reordered_pct -- 상품별 재 주문 비율
  , no_reordered_cnt/prd_total_cnt no_reordered_pct -- 상품별 재주문 하지 않은 비율
  , b.product_name, c.aisle_id, c.aisle, d.department_id, d.department
from order_prods_grp a, products b, aisles c, depts d 
where a.product_id = b.product_id 
and b.aisle_id = c.aisle_id
and b.department_id = d.department_id
)
-- end of with 절
-- group by를 aisle_id 별로 수행. 
select aisle_id, max(aisle) as aisle 
  , sum(reordered_cnt) aisle_reordered_cnt --상품 중분류별 재 주문 건수 
  , sum(no_reordered_cnt) aisle_no_reordered_cnt --상품 중분류별 재 주문하지 않은 건수. 
  , sum(reordered_cnt)/sum(a.prd_total_cnt) aisle_reordered_pct --상품 중분류별 재 주문 비율
  , sum(no_reordered_cnt)/sum(a.prd_total_cnt) aisle_no_reordered_pct -- 상품 중분류별 재 주문하지 않은 비율
  , sum(prd_total_cnt) aisle_total_cnt -- 상품 중분류별 건수
from order_prods_grp_detail a 
group by aisle_id
order by aisle_total_cnt desc limit 1000
--order by aisle_reordered_pct limit 1000 -- reordered_pct가 높은 순으로 정렬 

In [0]:
%sql 
-- 상품별 주문 reorder/no reorder 건수와 비율, 그리고 aisle 대비 reorder 건수 차이 비교
with 
-- 첫번째 임시 집합 order_prods_grp
order_prods_grp as
(
select product_id, 
  sum(case when reordered=1 then 1 else 0 end) as reordered_cnt,
  sum(case when reordered=0 then 1 else 0 end) as no_reordered_cnt,
  count(*) prd_total_cnt
from priors_trains -- trains, priors 
group by product_id -- group by product_id having total_cnt > 100
), 
-- 두번째 임시 집합 order_prods_grp_detail
order_prods_grp_detail as
(
select a.*, 
 reordered_cnt/prd_total_cnt reordered_pct, no_reordered_cnt/prd_total_cnt no_reordered_pct,
 b.product_name, c.aisle_id, c.aisle, d.department_id, d.department
from order_prods_grp a, products b, aisles c, depts d 
where a.product_id = b.product_id 
and b.aisle_id = c.aisle_id
and b.department_id = d.department_id
),
-- 세번째 임시 집합 order_aisle_grp_detail
order_aisle_grp_detail as
(
select aisle_id, max(aisle) aisle, 
  sum(reordered_cnt) aisle_reordered_cnt, 
  sum(no_reordered_cnt) aisle_no_reordered_cnt,
  sum(reordered_cnt)/sum(a.prd_total_cnt) aisle_reordered_pct,
  sum(no_reordered_cnt)/sum(a.prd_total_cnt) aisle_no_reordered_pct,
  sum(prd_total_cnt) aisle_total_cnt
from order_prods_grp_detail a 
group by aisle_id
)
-- end of with 절
select 
  a.*, 
  a.reordered_pct - b.aisle_reordered_pct as reordered_pct_diff_01,
  b.*
from order_prods_grp_detail a, order_aisle_grp_detail b
where a.aisle_id = b.aisle_id 
--and a.prd_total_cnt > 100
order by reordered_pct_diff_01 desc limit 1000

### 상품별 고유 사용자 건수 및 이전 주문 이후 현 주문까지 걸리는 일자 분석
* 상품별 고유 사용자가 많으면 여러 사용자가 주문을 하는 상품. 
* 상품별 주문 건수가 높고 고유 사용자 건수도 함께 높으면 다양한 고객으로 부터 많은 선택을 받는 상품. 일반적으로 주문 건수가 높으면 고유 사용자 건수는 상대적으로 낮음.
* 주문 건수가 작은 상품의 경우 서로 다른 개별 user 비율이 높아질 가능성 증대

In [0]:
%sql
describe priors_trains -- describe orders

In [0]:
%sql
drop table if exists order_all_prods

In [0]:
%fs
rm -r dbfs:/user/hive/warehouse/order_all_prods

In [0]:
%sql
-- orders 테이블을 여러 값으로 분석하기 위해서 orders 테이블과 조인. 
create table order_all_prods
as
select a.order_id, a.product_id, a.add_to_cart_order, a.reordered,
  b.user_id, b.eval_set, b.order_number, b.order_dow, b.order_hour_of_day,
  b.days_since_prior_order
from priors_trains a, orders b
where a.order_id = b.order_id

In [0]:
%sql
select 'priors_trains' gubun, count(*) from priors_trains 
union all
select 'order_all', count(*) from order_all_prods
union all 
select 'order', count(*) from orders

In [0]:
%sql
select * from order_all_prods limit 10

In [0]:
%sh
vmstat 3

In [0]:
%sql 
-- 개별 상품의 주문시 서로 다른 개별 user의 비율 추출. 주문 건수가 작은 상품의 경우 서로 다른 개별 user 비율이 높아질 가능성 증대. 서로 다른 개별 user 비율이 높으면 reordere될 확률 작아짐. reorder 예측이 데이터의 편차에 따라 편향성이 커질 우려.  
select a.*
  , prd_distinct_usr_cnt/prd_total_cnt as prd_usr_ratio -- 상품별 전체 건수 대비 고유 사용자 비율
from (
  select product_id
    , count(distinct user_id) prd_distinct_usr_cnt -- 상품별 고유 사용자 건수
    , count(*) prd_total_cnt 
    , avg(reordered) avg_reordered
  from order_all_prods
  group by product_id
) a 
where prd_total_cnt > 300 order by prd_usr_ratio desc limit 100

In [0]:
%sql
-- 상품별 주문 건수에 따라 상품의 개별 사용자 비율이 어떻게 변하는지 추출. 
with 
order_prods_grp as
-- product_id 레벨로 group by 하여 상품별 서로 다른 개별 사용자 비율을 추출한 결과에 product_name과 중분류명, 대분류명을 알기 위해 aisles와 dept로 조인. 
(
  select product_id
    , count(distinct user_id) prd_distinct_usr_cnt -- 상품별 고유 사용자 건수
    , count(*)  prd_total_cnt -- 상품별 전체 주문 건수 
    , count(distinct user_id)/count(*) as prd_usr_ratio -- 상품별 전체 건수 대비 고유 사용자 비율
  from order_all_prods
  group by product_id
)
-- end of with 절
select floor(prd_total_cnt/1000.0)*1000 as bin, avg(prd_usr_ratio) from order_prods_grp
group by floor(prd_total_cnt/1000.0)*1000 order by 1


In [0]:
%sql
with 
-- with 구문 첫번째 집합. product_id 레벨로 group by 하여 상품별 서로 다른 개별 사용자 비율을 추출한 결과에 상품명과 상품 중분류명 알기 위해 products와 aisles로 조인
order_prods_grp as
(
  select a.product_id, max(b.product_name) product_name
  , count(distinct user_id) as prd_distinct_usr_cnt -- 상품별 고유 사용자 건수
  , count(*)  as prd_total_cnt  -- 상품별 전체 주문 건수
  , count(distinct user_id)/count(*) as prd_usr_ratio -- 상품별 전체 건수 대비 고유 사용자 비율
  , avg(reordered) as avg_reordered 
  , max(c.aisle_id) as aisle_id, max(c.aisle) as aisle_name
  from order_all_prods a, products b, aisles c
  where a.product_id = b.product_id 
  and b.aisle_id = c.aisle_id
  group by a.product_id
),
-- with 구문 두번째 집합. 상품 중분류별 고유사용자 건수/비율. 상품 중분류명을 알기위해 aisles와 조인. 
order_aisles_grp as
(
select c.aisle_id, max(c.aisle) aisle_name
  , count(distinct a.user_id) aisle_distinct_usr_cnt -- 상품 중분류별 고유 사용자 건수
  , count(*)  aisle_total_cnt -- 상품 중분류별 전체 주문 건수
  , count(distinct a.user_id)/count(*) as aisle_usr_ratio -- 상품 중분류별 전체 건수 대비 고유 사용자 비율
from order_all_prods a, products b, aisles c
where a.product_id = b.product_id 
and b.aisle_id = c.aisle_id
group by c.aisle_id
),
-- with 구문 세번째 집합. 상품 중분류 별 개별 사용자 비율과 상품별 개별 사용자 비율 차이 추출. 
order_prd_alias_grp_diff as
(
select a.*, b.aisle_distinct_usr_cnt, b.aisle_total_cnt, b.aisle_usr_ratio, 
  a.prd_usr_ratio - b.aisle_usr_ratio as usr_ratio_diff
from order_prods_grp a, order_aisles_grp b
where a.aisle_id = b.aisle_id
) 
-- end of with 절
select * from order_prd_alias_grp_diff
/* usr_ratio_diff 를 histogram 형태로 추출. 

select floor(usr_ratio_diff/0.1)*0.1 as bin, count(*) as cnt
from order_prd_alias_grp_diff group by floor(usr_ratio_diff/0.1)*0.1
*/

상품별 평균/최소/최대 days_since_prior_order

In [0]:
%sql
select *
from orders a where user_id = 1 

In [0]:
%sql
-- 상품별 평균/최소/최대 days_since_prior_order
with 
order_prods_grp as
(
select product_id, 
    avg(days_since_prior_order) as prd_avg_prior_days 
    ,min(days_since_prior_order) as prd_min_prior_days 
    ,max(days_since_prior_order) as prd_max_prior_days
    ,count(*) prd_total_cnt
from order_all_prods group by  product_id)
-- end of with 구문
select * from order_prods_grp
/*
select floor(prd_avg_prior_days/1.0)*1 as bin, count(*) bin_cnt
from order_prods_grp group by floor(prd_avg_prior_days/1.0)*1 order by 1
*/

In [0]:
%sql
-- 상품별 평균/최소/최대 days_since_prior_order
-- instacart는 30일 마다 자동 주문 설정 기능이 있음. 
with 
order_prods_grp as
(
select product_id, 
    avg(days_since_prior_order) as prd_avg_prior_days 
    ,min(days_since_prior_order) as prd_min_prior_days 
    ,max(days_since_prior_order) as prd_max_prior_days
    ,count(*) prd_total_cnt
from order_all_prods group by  product_id)
-- end of with 구문
select floor(prd_max_prior_days/1.0)*1 as bin, count(*) bin_cnt
from order_prods_grp group by floor(prd_max_prior_days/1.0)*1 order by 1
/*
select floor(prd_avg_prior_days/1.0)*1 as bin, count(*) bin_cnt
from order_prods_grp group by floor(prd_avg_prior_days/1.0)*1 order by 1

select floor(prd_max_prior_days/1.0)*1 as bin, count(*) bin_cnt
from order_prods_grp group by floor(prd_max_prior_days/1.0)*1 order by 1

*/

상품별 평균 요일, 요일별 주문 건수

In [0]:
%sql
select * from order_all_prods where user_id = 1

In [0]:
%sql
-- 상품별 요일/시간 평균 값은 아님. 단순히 상품별로 주문된 order_dow/order_hour 값을 평균
-- 요일별 상품 주문 건수 
SELECT product_id, 
    avg(order_dow) prd_avg_order_dow,
    -- 요일 별 상품 주문 건수 
    sum(case when order_dow=0 then 1 end) prd_cnt_order_dow_0,
    sum(case when order_dow=1 then 1 end) prd_cnt_order_dow_1,
    sum(case when order_dow=2 then 1 end) prd_cnt_order_dow_2,
    sum(case when order_dow=3 then 1 end) prd_cnt_order_dow_3,
    sum(case when order_dow=4 then 1 end) prd_cnt_order_dow_4,
    sum(case when order_dow=5 then 1 end) prd_cnt_order_dow_5,
    sum(case when order_dow=6 then 1 end) prd_cnt_order_dow_6,
    avg(order_hour_of_day) prd_avg_order_hour_of_day
FROM 
    order_all_prods GROUP BY product_id

In [0]:
%sql
select order_dow, reordered, count(*) as cnt
from order_all_prods group by order_dow, reordered order by 1
/*
select order_dow, sum(case when reordered=1 then 1 else 0 end) reordered_cnt
,sum(case when reordered=0 then 1 else 0 end) as no_reordered_cnt
from order_all_prods group by order_dow, reordered order by 1
*/

In [0]:
%sql
select order_hour_of_day, reordered, count(*) as cnt
from order_all_prods group by order_hour_of_day, reordered order by 1